In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt

In [11]:
trainingPercent = 90
testingPercent = 10
epochs = 10000
learningRate = 0.01

In [12]:
def readData(fileName):
    df = pd.read_csv(fileName)
    return df

In [13]:
df = readData("insurance.csv")

In [14]:
def encodeData(df):
    sex = df.iloc[:,:].values
    labelEncoder = LabelEncoder()
    sex[:,1] = labelEncoder.fit_transform(sex[:,1])
    step1 = pd.DataFrame(sex)
    
    smoker = step1.iloc[:,:].values
    smoker[:,4] = labelEncoder.fit_transform(smoker[:,4])
    step2 = pd.DataFrame(smoker)
    
    region = step2.iloc[:,:].values
    region[:,5] = labelEncoder.fit_transform(region[:,5])
    return pd.DataFrame(region)

In [15]:
df = encodeData(df)

In [16]:
def normalizeData(df):
    df_norm = (df - df.mean()) / (df.max() - df.min())
    return df_norm,df.mean(),df.max(),df.min()

In [17]:
df,mean,maximum,minimum = normalizeData(df)
print(mean)

0       39.207025
1        0.505232
2       30.663397
3        1.094918
4        0.204783
5        1.515695
6    13270.422265
dtype: float64


In [18]:
print(maximum)

0       64.00000
1        1.00000
2       53.13000
3        5.00000
4        1.00000
5        3.00000
6    63770.42801
dtype: float64


In [19]:
print(minimum)

0      18.0000
1       0.0000
2      15.9600
3       0.0000
4       0.0000
5       0.0000
6    1121.8739
dtype: float64


In [20]:
def shuffleData(df):
    shuffled = df.sample(frac=1).reset_index(drop=True)
    return shuffled

In [21]:
df = shuffleData(df)

In [22]:
def getTrainingAndTesting(df,trainingPercent):
    percentData = int(math.ceil((len(df))*(trainingPercent*0.01)))
    trainingFeatures = df.iloc[0:percentData,0:6]
    trainingTargets = df.iloc[0:percentData,6:7]
    testingFeatures = df.iloc[percentData:df.shape[0],0:6]
    testingTargets = df.iloc[percentData:df.shape[0],6:7]
    return trainingFeatures,trainingTargets,testingFeatures,testingTargets

In [23]:
trF,trT,teF,teT = getTrainingAndTesting(df,trainingPercent)

In [24]:
reg = LinearRegression().fit(trF, trT)

In [25]:
reg.score(trF, trT)

0.7466437226760374

In [26]:
reg.coef_

array([[ 0.18647177, -0.00100898,  0.2049251 ,  0.03886722,  0.37976515,
        -0.01930108]])

In [27]:
reg.intercept_

array([0.00113973])

In [ ]:
finaleWeights = [0.00113973,0.18647177,-0.00100898,0.2049251 ,0.03886722,0.37976515,-0.01930108]